In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import json
import datetime
import math
from random import randint

In [2]:
import sklearn
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

## Importing data

In [3]:
matches = pd.read_csv("../Data/matches.csv")
matches.head()

,Date,Team_1,Team_2,Venue,MatchCode,GroundCode
0,2015-03-04,AFG,AUS,Away,3773,65
1,2012-08-25,AFG,AUS,Neutral,3437,156
2,2019-06-01,AFG,AUS,Neutral,4306,149
3,2014-03-01,AFG,BAN,Away,3623,190
4,2016-09-25,AFG,BAN,Away,3936,198


In [4]:
attributes = pd.read_csv("../Data/attributes.csv")
attributes.head()

,Full Name,Born,Bats,Bowls,Code
0,David A Warner,1986-10-27,Left Handed,Right Arm Leg Break,3637
1,Aaron James Finch,1986-11-17,Right Handed,Left Arm Orthodox,3845
2,Steven Peter Devereux Smith,1989-02-06,Right Handed,Right Arm Leg Break,3756
3,Glenn James Maxwell,1988-10-14,Right Handed,Right Arm Off Break,3969
4,James Peter Faulkner,1990-04-29,Right Handed,Left Arm Medium,3930


In [5]:
batsmen = open('../Data/batsmen.json',) 
batsmen_data = json.load(batsmen)

In [6]:
bowlers = open('../Data/bowlers.json',) 
bowlers_data = json.load(bowlers)

In [7]:
invtmap = open('../Data/invtmap.json',) 
invtmap_data = json.load(invtmap)

In [8]:
scorecard = open('../Data/scorecard.json',) 
scorecard_data = json.load(scorecard)

In [9]:
region = open('../Data/region.json',) 
region_data = json.load(region)

In [10]:
tmap = open('../Data/tmap.json',) 
tmap_data = json.load(tmap)

## Model 1

## Making the Database

In [11]:
def get_matches(team_1,team_2,date,num_years=5):
    matches_team=matches[matches['Team_1']==team_1]
    matches_team=matches_team[matches['Team_2']==team_2]
    matches_team1=matches[matches['Team_1']==team_2]
    matches_team1=matches_team1[matches['Team_2']==team_1]
    matches_team = pd.concat([matches_team,matches_team1],axis=0)
    matches_team['Date']= pd.to_datetime(matches_team['Date'])
    get_date = datetime.datetime.strptime(date, '%Y-%m-%d')
    min_date = datetime.datetime.strptime(str(get_date.year-num_years)+'-01'+'-01', '%Y-%m-%d')
    matches_team_latest = matches_team[matches_team['Date']>=min_date]
    matches_team_latest = matches_team_latest[matches_team_latest['Date']<date]
    matches_team_latest = matches_team_latest.sort_values(by='Date')
    
    matches_team_latest = matches_team_latest.reset_index()
    matches_team_latest=matches_team_latest.drop(['index'],axis=1)
    return matches_team_latest

In [12]:
def create_Dataset(Dataset):
    Dataset['Toss'] = ''
    Dataset['Fours_team1'] = ''
    Dataset['Fours_team2'] = ''
    Dataset['Sixes_team1'] = ''
    Dataset['Sixes_team2'] = ''
    Dataset['Strike_rate_team1'] = ''
    Dataset['Strike_rate_team2'] = ''
    Dataset['Maidens_team1'] = ''
    Dataset['Maidens_team2'] = ''
    Dataset['Wickets_team1'] = ''
    Dataset['Wickets_team2'] = ''
    Dataset['Economy_rate_team1'] = ''
    Dataset['Economy_rate_team2'] = ''
    Dataset['Score_1']=''
    Dataset['Score_2']=''
    Dataset['Result'] = ''
    
    MatchCodes = Dataset['MatchCode'].tolist()
    
    for i in range(len(MatchCodes)):
        matchdata = scorecard_data[str(MatchCodes[i])]
        #Dataset = Dataset.drop(columns=['GroundCode','Team_1','Team_2'])
        if (matchdata['TOSS']==matchdata['ORDER'][0]):
            toss = 1
        else:
            toss = 2
        #Batting 1
        batting1 = matchdata['BATTING1']
        Runs_team1 = []
        Balls_Played_team1 = []
        Fours_team1 = []
        Sixes_team1 = []
        Strike_rate_team1 = []
        for val in range(10):
            Runs_team1.append(batting1[val][2])
            Balls_Played_team1.append(batting1[val][3])
            Fours_team1.append(batting1[val][4])
            Sixes_team1.append(batting1[val][5])
            Strike_rate_team1.append(batting1[val][6])

        #Batting 2
        batting2 = matchdata['BATTING2']
        Runs_team2 = []
        Balls_Played_team2 = []
        Fours_team2 = []
        Sixes_team2 = []
        Strike_rate_team2 = []
        for val in range(10):
            Runs_team2.append(batting2[val][2])
            Balls_Played_team2.append(batting2[val][3])
            Fours_team2.append(batting2[val][4])
            Sixes_team2.append(batting2[val][5])
            Strike_rate_team2.append(batting2[val][6])

        #Bowling 1
        bowling1 = matchdata['BOWLING1']
        Overs_team1 = []
        Maidens_team1 = []
        Runs_given_team1 = []
        Wickets_team1 = []
        Economy_rate_team1 = []
        for val in range(10):
            if val<len(bowling1):
                Overs_team1.append(bowling1[val][1])
                Maidens_team1.append(bowling1[val][2])
                Runs_given_team1.append(bowling1[val][3])
                Wickets_team1.append(bowling1[val][4])
                Economy_rate_team1.append(bowling1[val][5])
            else:
                Overs_team1.append(-1)
                Maidens_team1.append(-1)
                Runs_given_team1.append(-1)
                Wickets_team1.append(-1)
                Economy_rate_team1.append(-1)

        #Bowling 2
        bowling2 = matchdata['BOWLING2']
        Overs_team2 = []
        Maidens_team2 = []
        Runs_given_team2 = []
        Wickets_team2 = []
        Economy_rate_team2 = []
        for val in range(10):
            if val<len(bowling2):
                Overs_team2.append(bowling2[val][1])
                Maidens_team2.append(bowling2[val][2])
                Runs_given_team2.append(bowling2[val][3])
                Wickets_team2.append(bowling2[val][4])
                Economy_rate_team2.append(bowling2[val][5])
            else:
                Overs_team2.append(-1)
                Maidens_team2.append(-1)
                Runs_given_team2.append(-1)
                Wickets_team2.append(-1)
                Economy_rate_team2.append(-1)
        #Scores
        score = matchdata['SCORES']
        if (score[0]>score[1]):
            result = 1
        elif (score[1]>score[0]):
            result = 2
        else:
            result = 'Tie'

        #insert into dataset
        Dataset['Toss'].iloc[i] = toss
        Dataset['Score_1'].iloc[i]=score[0]
        Dataset['Score_2'].iloc[i]=score[1]
        Dataset['Fours_team1'].iloc[i] = Fours_team1
        Dataset['Fours_team2'].iloc[i] = Fours_team2
        Dataset['Sixes_team1'].iloc[i] = Sixes_team1
        Dataset['Sixes_team2'].iloc[i] = Sixes_team2
        Dataset['Strike_rate_team1'].iloc[i] = Strike_rate_team1
        Dataset['Strike_rate_team2'].iloc[i] = Strike_rate_team2
        Dataset['Maidens_team1'].iloc[i] = Maidens_team1
        Dataset['Maidens_team2'].iloc[i] = Maidens_team2
        Dataset['Wickets_team1'].iloc[i] = Wickets_team1
        Dataset['Wickets_team2'].iloc[i] = Wickets_team2
        Dataset['Economy_rate_team1'].iloc[i] = Economy_rate_team1
        Dataset['Economy_rate_team2'].iloc[i] = Economy_rate_team2
        Dataset['Result'].iloc[i] = result
    
    Dataset = Dataset.drop(columns=['Date','GroundCode'])
    return Dataset
    

In [13]:
#Example Input
team_1 = 'AUS'
team_2 = 'IND' 
date = '2016-01-20'

In [14]:
Dataset=get_matches(team_1,team_2,date)
Dataset=create_Dataset(Dataset)

<ipython-input-11-29ebaa57a627>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  matches_team=matches_team[matches['Team_2']==team_2]
<ipython-input-11-29ebaa57a627>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  matches_team1=matches_team1[matches['Team_2']==team_1]
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [15]:
Dataset.head()

,Team_1,Team_2,Venue,MatchCode,Toss,Fours_team1,Fours_team2,Sixes_team1,Sixes_team2,Strike_rate_team1,Strike_rate_team2,Maidens_team1,Maidens_team2,Wickets_team1,Wickets_team2,Economy_rate_team1,Economy_rate_team2,Score_1,Score_2,Result
0,AUS,IND,Away,3278,1,"[5.0, 6.0, 7.0, 0.0, 0.0, 0.0, 3.0, 0.0, -1, -1]","[2.0, 7.0, 2.0, 1.0, 8.0, 1.0, 2.0, -1, -1, -1]","[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, -1, -1]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, -1, -1, -1]","[65.79, 85.48, 88.14, 42.11, 33.33, 54.55, 146...","[68.18, 77.94, 78.13, 72.73, 87.69, 87.5, 121....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1]","[2.0, 2.0, 0.0, 0.0, 2.0, 0.0, 0.0, -1, -1, -1]","[1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, -1, -1, -1]","[5.2, 5.3, 5.0, 6.29, 4.4, 4.5, 6.0, -1, -1, -1]","[5.19, 7.43, 5.13, 5.29, 5.0, 6.33, 3.8, -1, -...",260,261,2
1,AUS,IND,Home,3373,1,"[2.0, 1.0, 5.0, 5.0, 5.0, 2.0, 1.0, 0.0, 0.0, -1]","[7.0, 3.0, 1.0, 1.0, 3.0, 0.0, 0.0, 0.0, -1, -1]","[0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1]","[0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, -1, -1]","[75.0, 42.86, 88.37, 79.52, 94.74, 108.33, 88....","[82.88, 95.24, 64.29, 80.49, 126.67, 75.86, 15...","[0.0, 1.0, 0.0, 1.0, 0.0, 0.0, -1, -1, -1, -1]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, -1, -1, -1, -1]","[1.0, 2.0, 0.0, 2.0, 0.0, 0.0, -1, -1, -1, -1]","[1.0, 0.0, 3.0, 0.0, 0.0, 2.0, -1, -1, -1, -1]","[4.6, 5.8, 5.0, 4.9, 5.88, 7.5, -1, -1, -1, -1]","[5.7, 6.13, 5.48, 4.5, 4.33, 5.67, -1, -1, -1,...",269,270,2
2,AUS,IND,Home,3380,1,"[2.0, 5.0, 0.0, 3.0, 6.0, 1.0, 5.0, -1, -1, -1]","[1.0, 0.0, 0.0, 0.0, 1.0, 2.0, 1.0, 1.0, 0.0, ...","[1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, -1, -1, -1]","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, ...","[67.16, 93.48, 26.92, 73.24, 113.46, 130.0, 16...","[100.0, 25.0, 48.0, 0.0, 68.29, 66.67, 51.43, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1, -1]","[1.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1, -1, -1]","[1.0, 0.0, 3.0, 0.0, 0.0, 1.0, -1, -1, -1, -1]","[5.0, 3.0, 1.0, 1.0, 0.0, -1, -1, -1, -1, -1]","[4.6, 6.0, 6.1, 4.4, 6.57, 5.67, -1, -1, -1, -1]","[3.47, 4.9, 4.5, 4.5, 2.9, -1, -1, -1, -1, -1]",288,178,1
3,AUS,IND,Home,3386,1,"[7.0, 0.0, 1.0, 1.0, 2.0, 2.0, 2.0, 0.0, 0.0, ...","[1.0, 2.0, 3.0, 2.0, 1.0, 0.0, 1.0, 2.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, ...","[103.03, 14.29, 38.89, 47.62, 84.38, 84.85, 75...","[45.45, 93.33, 47.92, 77.78, 114.29, 28.57, 47...","[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1, -1]","[0.0, 1.0, 0.0, 0.0, 0.0, 2.0, 0.0, -1, -1, -1]","[2.0, 0.0, 0.0, 2.0, 1.0, 3.0, -1, -1, -1, -1]","[1.0, 2.0, 1.0, 0.0, 1.0, 2.0, 2.0, -1, -1, -1]","[3.7, 5.6, 4.5, 6.5, 5.1, 4.78, -1, -1, -1, -1]","[3.25, 6.25, 4.5, 3.5, 2.67, 1.8, 3.47, -1, -1...",252,165,1
4,AUS,IND,Away,3562,1,"[5.0, 8.0, 0.0, 10.0, 0.0, 1.0, 2.0, 0.0, 2.0,...","[1.0, 6.0, 6.0, 2.0, 0.0, 2.0, 1.0, 0.0, 1.0, ...","[0.0, 3.0, 0.0, 0.0, 0.0, 3.0, 0.0, 2.0, 0.0, ...","[0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[88.68, 91.14, 50.0, 103.66, 53.85, 134.78, 71...","[46.67, 89.36, 71.76, 86.67, 100.0, 86.36, 61....","[2.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1, -1, -1]","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, -1, -1, -1]","[0.0, 1.0, 1.0, 2.0, 1.0, 0.0, 2.0, -1, -1, -1]","[1.0, 2.0, 3.0, 0.0, 2.0, 1.0, 1.0, -1, -1, -1]","[5.86, 7.56, 8.0, 5.5, 3.5, 12.0, 5.67, -1, -1...","[3.8, 3.6, 5.88, 5.4, 3.88, 6.0, 3.0, -1, -1, -1]",304,232,1


### Data for model

In [16]:
def get_model_data(Dataset):
    venue_encoding = {
    
    "Home" : 0,
    "Away" : 1,
    "Neutral" : 2
    }
    Dataset.replace({'Venue':venue_encoding}, inplace=True)
    enc = OneHotEncoder(handle_unknown='ignore')
    enc_df = pd.DataFrame(enc.fit_transform(Dataset[['Result','Toss']]).toarray())
    Dataset = Dataset.join(enc_df)
    Dataset.rename(columns = {0: 'Win1',1: 'Win2',2: 'Toss1',3: 'Toss2'}, inplace = True)
    y = Dataset[['Result','Win1','Win2','Score_1','Score_2']]
    Dataset.drop(['Result','Score_1','Score_2'],inplace=True, axis=1)
    Dataset.drop(['Team_1','Team_2'],inplace=True, axis=1)
    temp=Dataset['Venue']
    Dataset.drop(['Win1','Win2'],inplace=True, axis=1)
    Dataset.drop(['Venue'],inplace=True, axis=1)
    Dataset['Venue']=''
    Dataset['Venue']=temp
    Dataset.drop(['Toss'],inplace=True, axis=1)
    Dataset['new']=''
    for i in range(len(Dataset)):
        l=[]
        for j in range(1,len(Dataset.columns)-4):
            if(type(Dataset.iloc[i][j])!=list):
                l.append(Dataset.iloc[i][j])
            else:
                for k in range(len(Dataset.iloc[i][j])):
                    l.append(Dataset.iloc[i][j][k])
        Dataset['new'].iloc[i]=l 
    
    
    X=Dataset[['MatchCode','new','Toss1','Toss2','Venue']]
    return X,y

In [17]:
X,y=get_model_data(Dataset)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [18]:
# Test Train Split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/5, random_state = 100)

In [19]:
y_train=y

In [20]:
X

,MatchCode,new,Toss1,Toss2,Venue
0,3278,"[5.0, 6.0, 7.0, 0.0, 0.0, 0.0, 3.0, 0.0, -1, -...",1.0,0.0,1
1,3373,"[2.0, 1.0, 5.0, 5.0, 5.0, 2.0, 1.0, 0.0, 0.0, ...",1.0,0.0,0
2,3380,"[2.0, 5.0, 0.0, 3.0, 6.0, 1.0, 5.0, -1, -1, -1...",1.0,0.0,0
3,3386,"[7.0, 0.0, 1.0, 1.0, 2.0, 2.0, 2.0, 0.0, 0.0, ...",1.0,0.0,0
4,3562,"[5.0, 8.0, 0.0, 10.0, 0.0, 1.0, 2.0, 0.0, 2.0,...",1.0,0.0,1
5,3563,"[7.0, 8.0, 6.0, 8.0, 7.0, 0.0, 0.0, -1, -1, -1...",1.0,0.0,1
6,3564,"[2.0, 2.0, 9.0, 2.0, 0.0, 12.0, 0.0, 3.0, 1.0,...",0.0,1.0,1
7,3565,"[1.0, 1.0, 3.0, 7.0, 1.0, 6.0, 0.0, 1.0, 1.0, ...",0.0,1.0,1
8,3568,"[3.0, 3.0, 13.0, 13.0, 2.0, 5.0, 0.0, 0.0, -1,...",0.0,1.0,1
9,3572,"[12.0, 9.0, 0.0, 2.0, 0.0, 7.0, 0.0, -1, -1, -...",0.0,1.0,1


### Predict_data

In [21]:
def allBatsmen(batting):
    batting_keys=batting.keys()
    performance={}
    for k in batting_keys:
        for j in batting[k]:
            performance.setdefault(k,[]).append([j[0],j[1][1],j[1][2],j[1][3],j[1][4],j[1][5]])

    performance_keys = performance.keys()
    
    for per in performance_keys:
        l = performance[per]
        l1=sorted(l,reverse=True)
        performance[per]=l1
        
    return performance

In [22]:
def allBowlers(bowling):
    bowling_keys=bowling.keys()
    performance={}
    for k in bowling_keys:
        for j in bowling[k]:
            performance.setdefault(k,[]).append([j[0],j[1][0],j[1][1],j[1][2],j[1][3],j[1][4]])

    performance_keys = performance.keys()
    
    for per in performance_keys:
        l = performance[per]
        l1=sorted(l,reverse=True)
        performance[per]=l1
        
    return performance

In [23]:
def getLatestPerf_Bat(perform,team_players, date, num_years=5):
    latest_performance={}
    perform_keys = perform.keys()
    for k in team_players:
        for i in perform[k]:
            d = datetime.datetime.strptime(date, '%Y-%m-%d')
            d1 = datetime.datetime.strptime(i[0], '%Y-%m-%d')
            min_date = datetime.datetime.strptime(str(d.year-num_years)+'-01'+'-01', '%Y-%m-%d')
            if(d1>=min_date and i[0]<date):
                latest_performance.setdefault(k,[]).append(i)
    return latest_performance

In [24]:
def getAvgBat(team,index):
    final_score=[]
    for k in team.keys():
        para=[]
        for j in team[k]:
            para.append(j[index])
        
        para1=pd.Series(para)
        f1=para1.ewm(com=0.5,ignore_na=True,min_periods=1).mean()
       
        f1=np.nanmean(f1)
        final_score.append(round(f1,3))
            
    return final_score

In [25]:
def getLatestPerf_Bowl(perform,team_players, date, num_years=5):
    latest_performance={}
    perform_keys = perform.keys()
    for k in team_players:
        for i in perform[k]:
            d = datetime.datetime.strptime(date, '%Y-%m-%d')
            d1 = datetime.datetime.strptime(i[0], '%Y-%m-%d')
            min_date = datetime.datetime.strptime(str(d.year-num_years)+'-01'+'-01', '%Y-%m-%d')
            if(d1>=min_date and i[0]<date):
                latest_performance.setdefault(k,[]).append(i)
    return latest_performance

In [26]:
def getAvgBowl(team,index):
    final_score=[]
    for k in team.keys():
        para=[]
        for j in team[k]:
            para.append(j[index])
            
        para1=[x if x!=-1 else np.nan for x in para]
        para1=pd.Series(para1)
        f1=para1.ewm(com=0.5).mean()
        f1=np.nanmean(f1)
        final_score.append(round(f1,3))
    
    return final_score

In [27]:
def adjust_length(l):
    if (len(l)<10):
        l.extend([-1] * (10 - len(l)))
    return l

In [28]:
def get_matches_for_test(matches):
    x = [randint(0, len(matches)) for p in range(0, 10)]
    match_codes=[]
    for i in x:
        match_codes.append(matches['MatchCode'].iloc[i])
    return match_codes

In [29]:
def get_testing_data(X_test,match_codes, scorecard_data, matches):
    for i in range(len(match_codes)):
        match_code=match_codes[i]
        scorecard=scorecard_data[str(match_code)]
        team_1_batsmen = [scorecard['BATTING1'][i][0] for i in range(10)]
        team_2_batsmen = [scorecard['BATTING2'][i][0] for i in range(10)]
        team_1_bowlers = [scorecard['BOWLING1'][i][0] for i in range(len(scorecard['BOWLING1']))]
        team_2_bowlers = [scorecard['BOWLING2'][i][0] for i in range(len(scorecard['BOWLING2']))]
        date=matches[matches['MatchCode']==match_code]['Date'].iloc[0]
        if (scorecard['TOSS']==scorecard['ORDER'][0]):
            toss = 1
        else:
            toss = 2
        team1_bat=getLatestPerf_Bat(bat,team_1_batsmen, date, 5)
        team2_bat=getLatestPerf_Bat(bat,team_2_batsmen, date, 5)
        team1_bowl=getLatestPerf_Bowl(bowl,team_1_bowlers, date, 5)
        team2_bowl=getLatestPerf_Bowl(bowl,team_2_bowlers, date, 5)

        row = adjust_length(getAvgBat(team1_bat,3))+adjust_length(getAvgBat(team2_bat,3))+\
        adjust_length(getAvgBat(team1_bat,4))+adjust_length(getAvgBat(team1_bat,4))+\
        adjust_length(getAvgBat(team2_bat,5))+adjust_length(getAvgBat(team2_bat,5))+\
        adjust_length(getAvgBowl(team1_bowl,1))+adjust_length(getAvgBowl(team2_bowl,1))+\
        adjust_length(getAvgBowl(team1_bowl,2))+adjust_length(getAvgBowl(team2_bowl,2))+\
        adjust_length(getAvgBowl(team1_bowl,4))+\
        adjust_length(getAvgBowl(team2_bowl,4))

        X_test['new'].iloc[i]=row
        X_test['Toss'].iloc[i]=toss
        
        score = scorecard_data[str(match_codes[i])]['SCORES']
        if (score[0]>score[1]):
            X_test['Result'].iloc[i] = 1
        elif (score[1]>score[0]):
            X_test['Result'].iloc[i] = 2
        else:
            X_test['Result'].iloc[i] = 0
       
        X_test['Score_1'].iloc[i]=score[0]
        X_test['Score_2'].iloc[i]=score[1]
        
    return X_test



In [30]:
match_test=get_matches_for_test(matches)

In [31]:
bat={}
bat=allBatsmen(batsmen_data)
print(len(bat))

2099


In [32]:
bowl={}
bowl=allBowlers(bowlers_data)
print(len(bowl))

1616


In [33]:
X_test=pd.DataFrame(match_test)
X_test.columns=['MatchCode']
X_test['Toss']=''
X_test['new']=''
X_test['Score_1']=''
X_test['Score_2']=''
X_test['Result']=''

In [34]:
X_test =get_testing_data(X_test,match_test, scorecard_data, matches)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 

In [35]:
y_test=X_test[['Score_1','Score_2','Result']]

In [36]:
X_train=X

In [37]:
enc = OneHotEncoder(handle_unknown='ignore')
enc_df = pd.DataFrame(enc.fit_transform(X_test[['Toss']]).toarray())
X_test = X_test.join(enc_df)
X_test.rename(columns = {0: 'Toss1',1: 'Toss2'}, inplace = True)
X_test

,MatchCode,Toss,new,Score_1,Score_2,Result,Toss1,Toss2
0,1436,1,"[3.639, 3.013, 0.583, 2.266, 1.702, -0.056, 1....",281,283,2,1.0,0.0
1,3391,1,"[2.271, 1.677, 4.209, 1.916, 1.813, 2.709, 1.8...",321,306,1,1.0,0.0
2,3524,2,"[3.566, 2.627, 1.345, 0.967, 3.801, 2.044, 1.2...",348,187,1,0.0,1.0
3,3510,2,"[3.014, 3.561, 2.154, 2.561, 2.153, 1.204, 1.7...",233,236,2,0.0,1.0
4,2300,1,"[4.056, 4.824, 3.788, 2.842, 1.808, 2.198, 1.2...",348,257,1,1.0,0.0
5,1759,2,"[3.043, 3.442, 2.626, 3.136, -1.0, 1.326, 1.09...",273,274,2,0.0,1.0
6,381,2,"[1.656, 2.062, 2.171, 2.234, 0.519, 1.467, 1.3...",175,132,1,0.0,1.0
7,530,1,"[1.368, 0.398, 1.35, 1.909, 1.44, 0.99, 0.968,...",218,219,2,1.0,0.0
8,421,2,"[5.343, 2.206, 2.967, 3.127, 1.366, 1.284, 0.3...",165,166,2,0.0,1.0
9,1200,1,"[3.66, 2.571, -0.228, 1.683, 1.898, 2.04, 0.48...",185,188,2,1.0,0.0


In [38]:
len(X['new'].iloc[0])

120

### Model 

In [39]:
x=np.array(X['new'])
x_train=list(x)

In [40]:
x_test=np.array(X_test['new'])
x_test=list(x_test)

In [41]:
def model_eval(model, x_train, y_train, target, x_test,y_test):
    model.fit(x_train, y_train[target])
    predictions = model.predict(x_test)
    
    rms = sklearn.metrics.mean_squared_error(predictions,y_test[target])**0.5
    print('RMS : %s' % '{0:.2%}'.format(rms))
    
    r2 = sklearn.metrics.r2_score(predictions,y_test[target])
    print('R2 : %s' % '{0:.2%}'.format(r2))
    return np.asarray(predictions)

In [42]:
def winner_pred(model, X_train, Y_train, x_test, y_test):
    pred1 = model_eval(model, X_train, y_train,['Score_1'],x_test, y_test)
    pred2 = model_eval(model, X_train, y_train,['Score_2'],x_test, y_test)

    pred = pred1 - pred2

    for i in range(len(pred)):
        if ((pred[i])>0):
            pred[i] = 1
        else:
            pred[i] = 2

    #print(pred)
    sum=0
    print("Model Accuracy is: ")
    for i in range(len(pred)):
        if(pred[i]==y_test['Result'].iloc[i]):
            sum=sum+1
    print(sum/len(pred))
    

In [43]:
len(y_train)

15

In [44]:
model = DecisionTreeRegressor()
winner_pred(model, x_train, y_train, x_test,y_test)

RMS : 7535.32%
R2 : -2492.74%
RMS : 13090.76%
R2 : -250.83%
Model Accuracy is: 
0.4


In [45]:
model = LinearRegression()
winner_pred(model, x_train, y_train, x_test,y_test)

RMS : 6948.35%
R2 : -3892.44%
RMS : 6615.50%
R2 : -477.48%
Model Accuracy is: 
0.4


In [46]:
model = RandomForestRegressor()
winner_pred(model, x_train, y_train, x_test,y_test)

RMS : 7268.98%
R2 : -23261.56%


<ipython-input-41-c2137f2e14d1>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train, y_train[target])
<ipython-input-41-c2137f2e14d1>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train, y_train[target])


RMS : 7728.36%
R2 : -12538.28%
Model Accuracy is: 
0.4


In [47]:
x_train=np.array(X_train['new'])
x_train=list(x_train)

In [48]:
x_test=np.array(X_test['new'])
x_test=list(x_test)

In [49]:
model = DecisionTreeRegressor()
winner_pred(model, x_train, y_train, x_test,y_test)

RMS : 7301.37%
R2 : -3103.53%
RMS : 7938.14%
R2 : -23203.99%
Model Accuracy is: 
0.4


## Model 2

In [50]:
def create_Dataset1(Dataset):
    Dataset['Toss'] = ''
    
    Dataset['Strike_rate_team1'] = ''
    Dataset['Strike_rate_team2'] = ''
    
    Dataset['Wickets_team1'] = ''
    Dataset['Wickets_team2'] = ''
    Dataset['Economy_rate_team1'] = ''
    Dataset['Economy_rate_team2'] = ''
    Dataset['Score_1']=''
    Dataset['Score_2']=''
    Dataset['Result'] = ''
    
    MatchCodes = Dataset['MatchCode'].tolist()
    
    for i in range(len(MatchCodes)):
        matchdata = scorecard_data[str(MatchCodes[i])]
        #Dataset = Dataset.drop(columns=['GroundCode','Team_1','Team_2'])
        if (matchdata['TOSS']==matchdata['ORDER'][0]):
            toss = 1
        else:
            toss = 2
        #Batting 1
        batting1 = matchdata['BATTING1']
        Runs_team1 = []
        Balls_Played_team1 = []
        Fours_team1 = []
        Sixes_team1 = []
        Strike_rate_team1 = []
        for val in range(10):
            Runs_team1.append(batting1[val][2])
            Balls_Played_team1.append(batting1[val][3])
            Fours_team1.append(batting1[val][4])
            Sixes_team1.append(batting1[val][5])
            Strike_rate_team1.append(batting1[val][6])

        #Batting 2
        batting2 = matchdata['BATTING2']
        Runs_team2 = []
        Balls_Played_team2 = []
        Fours_team2 = []
        Sixes_team2 = []
        Strike_rate_team2 = []
        for val in range(10):
            Runs_team2.append(batting2[val][2])
            Balls_Played_team2.append(batting2[val][3])
            Fours_team2.append(batting2[val][4])
            Sixes_team2.append(batting2[val][5])
            Strike_rate_team2.append(batting2[val][6])

        #Bowling 1
        bowling1 = matchdata['BOWLING1']
        Overs_team1 = []
        Maidens_team1 = []
        Runs_given_team1 = []
        Wickets_team1 = []
        Economy_rate_team1 = []
        for val in range(10):
            if val<len(bowling1):
                Overs_team1.append(bowling1[val][1])
                Maidens_team1.append(bowling1[val][2])
                Runs_given_team1.append(bowling1[val][3])
                Wickets_team1.append(bowling1[val][4])
                Economy_rate_team1.append(bowling1[val][5])
            else:
                Overs_team1.append(-1)
                Maidens_team1.append(-1)
                Runs_given_team1.append(-1)
                Wickets_team1.append(-1)
                Economy_rate_team1.append(-1)

        #Bowling 2
        bowling2 = matchdata['BOWLING2']
        Overs_team2 = []
        Maidens_team2 = []
        Runs_given_team2 = []
        Wickets_team2 = []
        Economy_rate_team2 = []
        for val in range(10):
            if val<len(bowling2):
                Overs_team2.append(bowling2[val][1])
                Maidens_team2.append(bowling2[val][2])
                Runs_given_team2.append(bowling2[val][3])
                Wickets_team2.append(bowling2[val][4])
                Economy_rate_team2.append(bowling2[val][5])
            else:
                Overs_team2.append(-1)
                Maidens_team2.append(-1)
                Runs_given_team2.append(-1)
                Wickets_team2.append(-1)
                Economy_rate_team2.append(-1)
        #Scores
        score = matchdata['SCORES']
        if (score[0]>score[1]):
            result = 1
        elif (score[1]>score[0]):
            result = 2
        else:
            result = 'Tie'

        #insert into dataset
        Dataset['Toss'].iloc[i] = toss
        Dataset['Score_1'].iloc[i]=score[0]
        Dataset['Score_2'].iloc[i]=score[1]
       
        Dataset['Strike_rate_team1'].iloc[i] = Strike_rate_team1
        Dataset['Strike_rate_team2'].iloc[i] = Strike_rate_team2
        
        Dataset['Wickets_team1'].iloc[i] = Wickets_team1
        Dataset['Wickets_team2'].iloc[i] = Wickets_team2
        Dataset['Economy_rate_team1'].iloc[i] = Economy_rate_team1
        Dataset['Economy_rate_team2'].iloc[i] = Economy_rate_team2
        Dataset['Result'].iloc[i] = result
    
    Dataset = Dataset.drop(columns=['Date','GroundCode'])
    return Dataset
    

In [51]:
Dataset1=get_matches(team_1,team_2,date)
Dataset1=create_Dataset1(Dataset1)

<ipython-input-11-29ebaa57a627>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  matches_team=matches_team[matches['Team_2']==team_2]
<ipython-input-11-29ebaa57a627>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  matches_team1=matches_team1[matches['Team_2']==team_1]
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [52]:
Dataset1.head()

,Team_1,Team_2,Venue,MatchCode,Toss,Strike_rate_team1,Strike_rate_team2,Wickets_team1,Wickets_team2,Economy_rate_team1,Economy_rate_team2,Score_1,Score_2,Result
0,AUS,IND,Away,3278,1,"[65.79, 85.48, 88.14, 42.11, 33.33, 54.55, 146...","[68.18, 77.94, 78.13, 72.73, 87.69, 87.5, 121....","[2.0, 2.0, 0.0, 0.0, 2.0, 0.0, 0.0, -1, -1, -1]","[1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, -1, -1, -1]","[5.2, 5.3, 5.0, 6.29, 4.4, 4.5, 6.0, -1, -1, -1]","[5.19, 7.43, 5.13, 5.29, 5.0, 6.33, 3.8, -1, -...",260,261,2
1,AUS,IND,Home,3373,1,"[75.0, 42.86, 88.37, 79.52, 94.74, 108.33, 88....","[82.88, 95.24, 64.29, 80.49, 126.67, 75.86, 15...","[1.0, 2.0, 0.0, 2.0, 0.0, 0.0, -1, -1, -1, -1]","[1.0, 0.0, 3.0, 0.0, 0.0, 2.0, -1, -1, -1, -1]","[4.6, 5.8, 5.0, 4.9, 5.88, 7.5, -1, -1, -1, -1]","[5.7, 6.13, 5.48, 4.5, 4.33, 5.67, -1, -1, -1,...",269,270,2
2,AUS,IND,Home,3380,1,"[67.16, 93.48, 26.92, 73.24, 113.46, 130.0, 16...","[100.0, 25.0, 48.0, 0.0, 68.29, 66.67, 51.43, ...","[1.0, 0.0, 3.0, 0.0, 0.0, 1.0, -1, -1, -1, -1]","[5.0, 3.0, 1.0, 1.0, 0.0, -1, -1, -1, -1, -1]","[4.6, 6.0, 6.1, 4.4, 6.57, 5.67, -1, -1, -1, -1]","[3.47, 4.9, 4.5, 4.5, 2.9, -1, -1, -1, -1, -1]",288,178,1
3,AUS,IND,Home,3386,1,"[103.03, 14.29, 38.89, 47.62, 84.38, 84.85, 75...","[45.45, 93.33, 47.92, 77.78, 114.29, 28.57, 47...","[2.0, 0.0, 0.0, 2.0, 1.0, 3.0, -1, -1, -1, -1]","[1.0, 2.0, 1.0, 0.0, 1.0, 2.0, 2.0, -1, -1, -1]","[3.7, 5.6, 4.5, 6.5, 5.1, 4.78, -1, -1, -1, -1]","[3.25, 6.25, 4.5, 3.5, 2.67, 1.8, 3.47, -1, -1...",252,165,1
4,AUS,IND,Away,3562,1,"[88.68, 91.14, 50.0, 103.66, 53.85, 134.78, 71...","[46.67, 89.36, 71.76, 86.67, 100.0, 86.36, 61....","[0.0, 1.0, 1.0, 2.0, 1.0, 0.0, 2.0, -1, -1, -1]","[1.0, 2.0, 3.0, 0.0, 2.0, 1.0, 1.0, -1, -1, -1]","[5.86, 7.56, 8.0, 5.5, 3.5, 12.0, 5.67, -1, -1...","[3.8, 3.6, 5.88, 5.4, 3.88, 6.0, 3.0, -1, -1, -1]",304,232,1


In [53]:
def get_testing_data1(X_test,match_codes, scorecard_data, matches):
    for i in range(len(match_codes)):
        match_code=match_codes[i]
        scorecard=scorecard_data[str(match_code)]
        team_1_batsmen = [scorecard['BATTING1'][i][0] for i in range(10)]
        team_2_batsmen = [scorecard['BATTING2'][i][0] for i in range(10)]
        team_1_bowlers = [scorecard['BOWLING1'][i][0] for i in range(len(scorecard['BOWLING1']))]
        team_2_bowlers = [scorecard['BOWLING2'][i][0] for i in range(len(scorecard['BOWLING2']))]
        date=matches[matches['MatchCode']==match_code]['Date'].iloc[0]
        if (scorecard['TOSS']==scorecard['ORDER'][0]):
            toss = 1
        else:
            toss = 2
        team1_bat=getLatestPerf_Bat(bat,team_1_batsmen, date, 5)
        team2_bat=getLatestPerf_Bat(bat,team_2_batsmen, date, 5)
        team1_bowl=getLatestPerf_Bowl(bowl,team_1_bowlers, date, 5)
        team2_bowl=getLatestPerf_Bowl(bowl,team_2_bowlers, date, 5)

        row = adjust_length(getAvgBat(team2_bat,5))+adjust_length(getAvgBat(team2_bat,5))+\
        adjust_length(getAvgBowl(team1_bowl,2))+adjust_length(getAvgBowl(team2_bowl,2))+\
        adjust_length(getAvgBowl(team1_bowl,4))+\
        adjust_length(getAvgBowl(team2_bowl,4))

        X_test['new'].iloc[i]=row
        X_test['Toss'].iloc[i]=toss
        
        score = scorecard_data[str(match_codes[i])]['SCORES']
        if (score[0]>score[1]):
            X_test['Result'].iloc[i] = 1
        elif (score[1]>score[0]):
            X_test['Result'].iloc[i] = 2
        else:
            X_test['Result'].iloc[i] = 0
       
        X_test['Score_1'].iloc[i]=score[0]
        X_test['Score_2'].iloc[i]=score[1]
        
    return X_test



In [54]:
X,y=get_model_data(Dataset1)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [55]:
X_train=X

In [56]:
match_test=get_matches_for_test(matches)

In [57]:
X_test=pd.DataFrame(match_test)
X_test.columns=['MatchCode']
X_test['Toss']=''
X_test['new']=''
X_test['Score_1']=''
X_test['Score_2']=''
X_test['Result']=''

In [58]:
X_test =get_testing_data1(X_test,match_test, scorecard_data, matches)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 

In [59]:
y_test=X_test[['Score_1','Score_2','Result']]

In [60]:
y_train=y

### Model

In [61]:
x=np.array(X['new'])
x_train=list(x)

In [62]:
x=np.array(X_test['new'])
x_test=list(x)

In [63]:
model = DecisionTreeRegressor()
winner_pred(model, x_train, y_train, x_test,y_test)

RMS : 4634.76%
R2 : -1253.05%
RMS : 4606.84%
R2 : -3.42%
Model Accuracy is: 
0.8


In [64]:
model = RandomForestRegressor()
winner_pred(model, x_train, y_train, x_test,y_test)

<ipython-input-41-c2137f2e14d1>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train, y_train[target])
<ipython-input-41-c2137f2e14d1>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train, y_train[target])


RMS : 4825.58%
R2 : -9078.21%
RMS : 5790.76%
R2 : -3758.96%
Model Accuracy is: 
0.8


In [65]:
model = LinearRegression()
winner_pred(model, x_train, y_train, x_test,y_test)

RMS : 5186.10%
R2 : -752.81%
RMS : 7847.53%
R2 : -739.20%
Model Accuracy is: 
0.8
